# The solution of the Anderson impurity model (AIM) with iterated perturbation theory (IPT)

In this tutorial we will solve the Anderson impurity model with the iterated perturbation theory, plot the spectral function and extract the quasiparticle properties from the self-energy for several system parameters.

In [ ]:
from triqs.gf import Gf, MeshImFreq, MeshReFreq, Omega, Flat
from triqs.gf import inverse, make_gf_from_fourier
from math import pi
import numpy as np

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib as mpl
mpl.rcParams['figure.dpi']=100 
from triqs.plot.mpl_interface import oplot, plt

We would like to calculate the properties of a half-filled Anderson impurity model 

$$
H = H_\text{atom} + H_\text{bath} + H_\text{hybridization}
$$

$$
H_\text{atom} = Un^c_\uparrow n^c_\downarrow + (\varepsilon_0 - \mu) (n^c_\uparrow + n^c_\downarrow)
$$

$$
H_\text{bath} = \sum\limits_{l\sigma}{\tilde{\varepsilon}_l a_{l\sigma}^\dagger}a_{l\sigma}
$$

$$
H_\text{hybridization} = \sum\limits_{l\sigma}{V_l a_{l\sigma}^\dagger}c_\sigma + V^*_l c_\sigma^\dagger a_{l\sigma}
$$

with a flat bath, which has a half-bandwidth of $D=10$ (see first Tutorial). The inverse temperature should be $\beta=100$. The bath is hybridized with an impurity of energy level $\varepsilon=0$ and hybridization constant $V=2$. The value of the Coulomb interaction should be $U=2$.

In [ ]:
# Please give the parameters here.
# Parameters
D     = 10.0
U     = 10
eps_0 = 0.
beta  = 100.0
V     = 2.

As a first step, we have to construct the Green function of the Weiss field ${\cal G}_0(i\omega_n)$ of shape $(1,1)$ with the appropriate Matsubara frequency mesh with $n_{i\omega}=1000$.

In [ ]:
# Please construct the mesh and the Green function here.
n_iw = 1000
iw_mesh = MeshImFreq(beta=beta, statistic='Fermion', n_iw=n_iw)
G0_iw = Gf(mesh=iw_mesh, target_shape=[1,1])

For the non-interacting problem the chemical potential $\mu=0$ at half-filling.

In [ ]:
# Please give the chemical potential here.
mu = 0.

In [ ]:
# Please fill the Weiss field with the appropriate values.
# Then calculate and print the filling of this Weiss field.
G0_iw << inverse(Omega - eps_0 + mu - V**2 * Flat(D))
print("density of G0: %s"%G0_iw.density())

For the calculation of the self-energy in IPT we first perform the Fourier transform of ${\cal G}_0(i\omega_n)$ to ${\cal G}_0(\tau)$.

In [ ]:
# Perform the Fourier transform here.
G0_tau = make_gf_from_fourier(G0_iw)

Then we calculate the second-order self-energy in imaginary frequency ${\cal G}_0(\tau)^3$.

In [ ]:
# Perform the calculation here.
Sigma_tau = U**2 * G0_tau * G0_tau * G0_tau

As a last step, we Fourier transform back to imaginary frequencies and add the first order term $U/2$.

In [ ]:
# Fourier transform back and add the first order.
Sigma_iw = U / 2. + make_gf_from_fourier(Sigma_tau)

In order to calculate the Green function of the impurity model, we first fill the Weiss field with the updated chemical potential $\mu = U/2$.

In [ ]:
# Please fill the Weiss field here.
mu = U / 2.
G0_iw << inverse(Omega - eps_0 + mu - V**2 * Flat(D))

Next, we construct the Green function by using the `copy()` function on the Weiss field and fill it using the Dyson equation

$$
G(i\omega_n)^{-1} = \cal{G}_0(i\omega_n)^{-1} - \Sigma(i\omega_n).
$$

We also check the filling here.

In [ ]:
# Please construct G(iw_n) and execute the Dyson equation.
G_iw = G0_iw.copy()
G_iw << inverse(inverse(G0_iw) - Sigma_iw)
print("density of G: %s"%G_iw.density())

For illustration purposes we want to obtain the analytic continuation of $G(i\omega_n)$ by using Pade. We first construct a Green function with a real-frequency mesh with a frequency window of $(-20,20)$ and $n_\omega=1000$ frequencies before using the `set_from_pade` method.

In [ ]:
# Please perform the analytic continuation here.
# Get real axis function with Pade approximation
G_w = Gf(mesh=MeshReFreq(window = (-20.0,20.0), n_w=1000), target_shape=[1,1])
G_w.set_from_pade(G_iw, 100, 0.01)

In [ ]:
# Please plot the spectral function here.
oplot(-G_w.imag/pi, linewidth=3, label=rf"$\beta$ = {beta:.2f}")
plt.xlim(-20,20)
plt.ylim(0,)
plt.ylabel("")
plt.show()

In order to get more insight into the physics, we also plot the real and imaginary parts of the self-energy, respectively, as a function of the Matsubara frequency for the lowest frequencies.

In [ ]:
# Please plot here the real part.
oplot(Sigma_iw.real, '-o', label='')
plt.xlim(0,4)

plt.ylabel(r'Re $\Sigma(i\omega_n)$')
plt.show()

In [ ]:
# Please plot here the imaginary part.
oplot(Sigma_iw.imag, '-o', label='')
plt.xlim(0,4)

plt.ylabel(r'Im $\Sigma(i\omega_n)$')
plt.show()

What is the physical interpretation of the spectral function given, particularly w.r.t. the discussion in the lecture about conductance in quantum dots? Please perform another calculation with $U=10$. What is the difference?

### <i class="fa fa-gear fa-x" style="color: #186391"></i> Exercise 1
We now want to gain a more quantitative understanding. For this, we want to extract the quasiparticle weight $Z$, the scattering rate $\gamma$, as well as the derived quantities effective mass $m^*/m=Z^{-1}$ (particularly simple for a *local* self-energy) and quasiparticle lifetime $\tau=\left(Z\gamma\right)^{-1}$. This can be done directly from the self-energy on the Matsubara axis:

$$
Z = \left[1 - \frac{\partial\text{Im }\Sigma(i\omega_n)}{\partial\omega_n}\Biggr|_{\omega_n\rightarrow 0} \right]^{-1},
$$
$$
\gamma = -\text{Im }\Sigma(i\omega_n)\biggr|_{\omega_n\rightarrow 0}.
$$

Calculate these quantities by first extracting the data from Im $\Sigma(i\omega_n)$ via the `data` member and then using numpy routines (`numpy.polyfit`) to fit the function for small positive frequencies, e.g., via a polynom of degree 3 for the first 10 frequencies. This procedure is sometimes called "poor man's analytic continuation". Check the quality of the fit by plotting the fit over the first frequencies. Using this polynom, then, calculate all the parameters above for several values of the interaction $U=0.001,2,4,6,8,10$. How do they behave?

### <i class="fa fa-gear fa-x" style="color: #186391"></i> Exercise 2
Carry out the same analysis for two temperature cuts (several values of $\beta$): one for $U=2$ and one for $U=10$.

In [ ]:
# extract data from Sigma
Im_Sigma_iw_data = Sigma_iw[0,0].imag.data[n_iw:]

In [ ]:
# extract Matsubara frequencies
iw_data = []
for iw in iw_mesh:
    if iw.imag > 0.:
        iw_data.append(iw.imag)

In [ ]:
# make fit of degree 2 with 3 frequencies
n_iw_fit = 10
degree = 3
fit = np.poly1d(np.polyfit(iw_data[:n_iw_fit], Im_Sigma_iw_data[:n_iw_fit], degree))

In [ ]:
# plot the fit and the frequencies
# continous frequencies for plotting fit
iw_cont = (2. * np.linspace(0,20) + 1) * pi / beta

plt.plot(iw_data[:n_iw_fit], Im_Sigma_iw_data[:n_iw_fit], 'o', color='red', label='used for fit')
plt.plot(iw_data[n_iw_fit:], Im_Sigma_iw_data[n_iw_fit:], 'o', color='blue', label='all frequencies')
plt.plot(iw_cont, fit(iw_cont), '-', color='green', label='fit')

plt.xlim(0,4)
plt.xlabel(r'$\omega_n$')

plt.legend(loc=1)

plt.ylabel(r'Im $\Sigma(i\omega_n)$')
plt.show()

In [ ]:
# extract quasiparticle parameters
gamma = -fit(0.)
print("scattering rate gamma: %s"%gamma)
Z = 1. / (1. - fit.deriv(1)(0.))
print("quasiparticle weight Z: %s"%Z)
m_eff = 1. / Z
print("effective mass m_eff: %s"%m_eff)
tau = 1. / (Z * gamma)
print("quasiparticle lifetime: %s"%tau)

In [ ]:
# looping over parameters for constant beta
Us = [0.001, 2., 4., 6., 8., 10.]
gammas = []
Zs = []
m_effs = []
taus = []

for U in Us:
    mu = 0.
    G0_iw << inverse(Omega - eps_0 + mu - V**2 * Flat(D))
    
    Sigma_tau = U**2 * G0_tau * G0_tau * G0_tau
    Sigma_iw = U / 2. + make_gf_from_fourier(Sigma_tau)

    Im_Sigma_iw_data = Sigma_iw[0,0].imag.data[n_iw:]
    fit = np.poly1d(np.polyfit(iw_data[:n_iw_fit], Im_Sigma_iw_data[:n_iw_fit], degree))

    gamma = -fit(0.)
    gammas.append(gamma)
    Z = 1. / (1. - fit.deriv(1)(0.))
    Zs.append(Z)
    m_eff = 1. / Z
    m_effs.append(m_eff)
    tau = 1. / (Z * gamma)
    taus.append(tau)

In [ ]:
plt.plot(Us, gammas, '-o')

plt.xlim(0,)
plt.ylim(0,)
plt.xlabel('U')
plt.ylabel(r'$\gamma$')
plt.show()

In [ ]:
plt.plot(Us, Zs, '-o')

plt.xlim(0,)
plt.ylim(0,)
plt.xlabel('U')
plt.ylabel(r'Z')
plt.show()

In [ ]:
plt.plot(Us, m_effs, '-o')

plt.xlim(0,)
plt.ylim(0,)
plt.xlabel('U')
plt.ylabel(r'$m^*/m$')
plt.show()

In [ ]:
plt.plot(Us, taus, '-o')

plt.xlim(0,)
plt.ylim(0,1000)
plt.xlabel('U')
plt.ylabel(r'$\tau$')
plt.show()

Similarly for Exercise 2 - however, here, we have to adapt the meshes since they are at *different temperatures* now.